# Clases (Programación orientada a objetos)

## Objetos y Clases en POO

Las **clases** definen unas propiedades mediante unas varibles internas
(que se denominan generalmente **atributos**) y las capacidades y
comportamiento mediante funciones también internas (que se denominan
genralmente **métodos**) 

Un **objeto** es una variable
creada a partir de una clase, o **instanciada**, y representa un caso
particular dentro del conjunto de posibles instancias de una clase (de la misma
forma que podemos considerar al número 7 como una instancia particular de la
clase "Números Enteros"), y el gato "Romeo" (en foto adjunta) es una instancia,
o caso cocreto, de la clase "Gatos".

![El gato Romeo](./art/gato.jpg)

Las clases se definen en Python con la palabra reservada `class`. En
teoría no hace falta nada más para crear una clase que su nombre; así,
la clase más sencilla que podemos pensar es:

In [7]:
class X:
    pass

y, aunque sea poco útil, funciona. Podemos
instanciar un objeto a partir de esta clase, simplemente hay que
usar el nombre de la clase como si fuera una función (El convenio
es usar nombres en [CamelCase](https://es.wikipedia.org/wiki/CamelCase) para las clases y minúsculas para las variables, modulos y
paquetes):

In [8]:
x = X()
print(x)

Podemos asignar dinámicamente atributos. Por ejemplo, imaginemos
que necesitamos una nueva clase, que nos permita almacenar
coordenadas geográficas. Podiamos crear atributos `latitud`
y `longitud` dinámicamente en el objeto `x` creado anteriormente:

In [9]:
x.latitud = 28.4779
x.longitud = -16.3118
print(x.latitud, x.longitud)

28.4779 -16.3118


Pero esto no es ni elegante, ni cómodo, ni *pitónico*. En la mayor parte de los
lenguajes orientados a objetos es obligatorio definir los atributos que
puede tener un objeto. En Python no es necesario, pero sí es conveniente
tener centralizado la creación y asignación de estos atributos. Para
eso podemos definir un método con un nombre especial, `__init__`, que
es invocado inmediatemente después de creado el objeto (por lo que
no es técnicamente el constructor, sino más bien inicializador, pero
mucha gente se refiere a esta función como el constructor). Ya puestos,
vamos a darle a nuestra clase un nuevo nombre y un poco más de empaque:

In [10]:
class Point:

    def __init__(self, lat, lng):
        self.latitud = lat
        self.longitud = lng

        
x = Point(28.4779, -16.3118)
print(x.latitud, x.longitud)

28.4779 -16.3118


Un poco por intuición, podemos suponer lo que hace Python
cuando se crea la variable `x` a partir de la clase `Point` (a partir
de ahora, *instanciar*). En primer lugar se crea el objeto.
Inmediatamente a continuación, como hemos visto, se llama al
inicializador, y parece que tiene sentido suponer que los dos parámetros
que usamos al crear al objeto son los mismos valores que se pasan
al método inicializador con los nombres `lat` y `lng`, pero
¿De dónde sale el primer parámetro, `self`? ¿Y qué representa?

**Nota**: Para programadores de C++, Java, C#

> En C++, Java o C# acceden a este mismo objeto propio
> usando la variable *mágica* `this`, y muchas veces ni
> se utiliza porque se considera implícita. En Python se prefirió esta
> forma por considerarla más explicita. De igual manera, los
> atributos dentro de la función tienen que venir precedidos por el
> `self.`, no hay alias mágicos para los atributos de la
> instancia. En este caso, para evitar la ambigüedad.

Empezemos por la segunda pregunta: `self` representa al propio
objeto recién creado. Además, podemos adelantar que este primer
parámetro es lo que diferencia a las funciones que ya conocíamos
de los métodos; un método siempre tiene como primer parámetro
la instancia sobre la que está siendo ejecutado.

**Nota de implmentación**: Cuando definimos un método (en este caso `__init__`, pero vale para
cualquiera), se generan en realidad dos versiones del mismo. La
versión que se asocia a la clase, y la versión que se asocia
al objeto. Podemos verlo usando la función `id`, que nos muestra
las identidades de los objetos, o la funcion `is`:

In [11]:
class Point:
    def __init__(self, lat, lng):
        self.latitud = lat
        self.longitud = lng

x = Point(28.4779, -16.3118)
print(id(Point.__init__), id(x.__init__))
Point.__init__ is x.__init__

3024807084 3051684236


False

Es más, si vemos la representacion de ambas funciones (usando la función [`repr`](https://docs.python.org/3/library/functions.html#repr), ya vemos que son distintas::

In [12]:
print(repr(Point.__init__))
print(repr(x.__init__))

<function Point.__init__ at 0xb44ae4ac>
<bound method Point.__init__ of <__main__.Point object at 0xb444db6c>>


Una de ellas, la correspondiente a la clase `Point` se describe
como *unbound method* (método no vinculado o libre), mientras
que la correspondiente a la instancia se describe
como *bound method* (método vinculado o ligado). ¿Que significa
esto? Veamoslo con calma:

Cuando se intenta acceder a la función `__init__` desde la instancia, la
clase prepara otra version especial de `__init__`, una en la que el
primer parámetro ya ha sido "rellenado", por así decirlo, con la
propia instancia, y es éste el que se vincula a la instancia. De ahí
que se pase de un método libre (el de la clase, que acepta tres
parámetros) a un método vinculado (En la que el primer parámetro ya
esta definido y acepta, por tanto, solo dos). Esta es la
única diferencia entre los dos métodos.

Quizá ayude verlo con otro método más normal. Definamos un método que
nos indique si un determinado punto está por encima o por debajo del
Ecuador. Nuestro código queda así:

In [13]:
class Point:
    def __init__(self, lat, lng):
        self.latitud = lat
        self.longitud = lng

    def esta_en_hemisferio_sur(self):
        return self.latitud < 0 

Si usamos esta definición, vemos que podemos invocar cada una de
las funciones, la de la clase y la de la instancia, si tenemos
el cuidado de pasarle a cada una de ellas los parámetros que
necesita:

In [14]:
x = Point(28.4779, -16.3118)
print('x Está en el hemisferio sur (bound):', x.esta_en_hemisferio_sur())
print('x Está en el hemisferio sur (unbound)', Point.esta_en_hemisferio_sur(x))

x Está en el hemisferio sur (bound): False
x Está en el hemisferio sur (unbound) False


En resumen, cuando definimos métodos para una clase, tenemos que
**reservar el primer parámetro para el propio objeto**.  A la hora de
llamar al método desde la instancia, los engranajes internos de Python
ya se ocupan de poner el valor correcto como primer parámetro. La
tradición y la costumbre marcan que este primer parámetro se llame
`self`, pero en realidad no existe ninguna obligación de hacerlo
impuesta por el lenguaje. Esta convención, sin embargo, es de las más
respetadas dentro de la comunidad, y es poco recomendable usar 
otra.

### Métodos y/o atributos privados

En Python no existe el concepto de métodos o atributos privados. En
vez de eso, existe una convención de uso, por la cual si un atributo o
método empieza con el carácter subrayado, ha de entenderse que es de
uso interno, que no deberías jugar con él a no ser que sepas muy bien
lo que estás haciendo, y que si en un futuro tu código deja de
funcionar porque ese atributo o método ha desaparecido, no puedes
culpar a nadie más que a ti mismo. En resumen, se supone que todos
somos adultos responsables. Aunque parezca una locura, funciona.

## Métodos mágicos


## Polimorfismo

### Herencia de clases

Con esto tenemos un sistema bastante potente, una forma de agrupar en
una sola entidad la estructura de datos y el código asociado  con la
misma, así que podemos tener nuestros propios tipos de datos. Pero en
el mundo OOP (*Object Oriented Programming*), para poder hablar de clases y objetos  con propiedad
es necesario que haya algún tipo de herencia. La herencia nos permite
definir una clase refinando o modificando otra (herencia simple) u
otras (herencia múltiple). Veamos el caso de la herencia simple para
empezar.

Si decimos que una clase `A` **deriva** o **hereda** de una clase `B` (o
también que la clase `B` es una **superclase** de `A`), lo que queremos
decir es que la clase `A` dispondrá de todos los atributos y métodos de
la clase `B`, de entrada, aunque luego puede añadir más atributos o
métodos y modificar (o incluso borrar, pero está muy mal visto) los
atributos o métodos que ha heredado.

La forma de expresar esta herencia en Python es:

In [15]:
class B:
    pass

class A(B):
    pass

En este caso, la clase `A` hereda o deriva de de `B`.

Como los objetos instanciados de `A` tienen los mismos atributos y
métodos que `B`, pueden (o quizá deben, ya que nada lo impide
excepto el sentido común) ser usados en cualquier sitio donde se use
un objeto  instaciado de `B`. Esto implica que hay una relación entre `A` y `B`
del tipo *es un tipo de*, es decir, donde `B` es una
generalidad de `A`, o quiza `A` sea una particularidad de `B`,
depende del punto de vista. 

En otras palabras,  todo objeto `a` instanciado de `A` debería ser
también un `B` (pero no  necesariamente al revés).

¿Para que sirve la herencia? Por encima de todo, para reducir el
tamaño del código evitando repeticiones. Si organizamos las herencias
correctamente en jerarquías, de más genéricas a más específicas,
podemos compatibilizar el código común de las primeras con el más
específico de las últimas. Además, y no es un beneficio baladí,
simplifica los usos de los objetos instanciados, de forma que podemos
tratarlos a todos como si fueran del mismo tipo, aún siendo
diferentes.

En la herencia múltiple, una clase puede heredar de más de una clase
base. El problema en estos casos es que puede haber conflictos si se
definen, en ramas distintas, un mismo método o atributo (llamado
herencia en diamante). En python hay un sistema de ordenación de las
clases, relativamente complicado, pero que asegura que no haya
ambigüedad con respecto a cual de las dos versiones del atributo o
método se va a usar.

### Sobreescritura o reescritura de métodos

En el caso de que la clase modifique uno de los métodos que ha
heredado, se dice que ha **reescrito** o **sobreescrito** (*override*)
el  método.

Como hemos visto, `A` puede sobreescribir un metodo `f` de `B`,
pero eso no afecta al resto del código. Si tenemos una lista con
objetos de tipo `A` y de tipo `B` mezclados, podemos invocar sin
miedo el método `f()` en cada uno de ellos, con la seguridad de que
en cada caso se invocará al método adecuado. Esta capacidad se llama
**polimorfismo** (del griego *múltiples formas*. Quizá el nombre no sea
el más adecuado, porque más que muchas formas, es más la misma forma, con
diferentes contenidos).

En el ejemplo de la lista de objetos derivados de `A` y
`B`, si no tuvieramos herencia (y polimorfismo) tendríamos que
implementar un `if` para poder distinguir entre los dos tipos, y
luego llamar a la versión correspondiente de cada método `f`.
además,  tendremos que revisar  la sentencia `if` si se nos ocurre
incluir una nueva clase `C`.

¿Que pasa si la clase `A` sobreescribe un método de `B`, pero aún
así ha de invocarlo? En realidad es un caso muy común, en el que la
clase `A` quiere hacer lo mismo que la clase `B` y *un poquito
más*. Hay una función `super` que nos ayuda a
invocar el código de la clase (o clases) de la que derivamos:

In [16]:
class B:
    def f(self):
        print('Llamando a f en clase B')
        
class A(B):
    def f(self):
        super().f()
        print('Llamando a f en clase A')
        
a = A()
a.f()

Llamando a f en clase B
Llamando a f en clase A


    >>> class A(B):
    ...     def f(self, arg):
    ...         super(A, self).f(arg)
    ...
    >>>

En Python 2.7 también podemos usar `super`, aunque la sintaxis es un poquito 
más rebuscada, hay que llamar a super con dos argumentos:

 - La clase en la que estamos definiendo el método
 - La instancia, es decir, el parámetro self 

In [17]:
class B:
    def f(self):
        print('Llamando a f en clase B')
        
class A(B):
    def f(self):
        super(A, self).f()
        print('Llamando a f en clase A')
        
a = A()
a.f()

Llamando a f en clase B
Llamando a f en clase A


Esta sintaxis más rebuscada también funciona en Python 3, así que si queremos podemos
usarla para tener código que funcione igual en las dos ramas. Si sabemos que nuestro
programa solo va a ejecutarse en Python 3 podemos usar la sintaxis más sencilla.

### Funciones útiles para tratar con objetos y clases

Si tenemos un objeto y queremos saber si es una instancia de  una
clase en particular, o de alguna de sus subclases , podemos usar la función `isinstance(objeto, clase)`, que nos devolverá verdadero si es así. Si queremos
saber si una clase deriva de otra podemos usar
la funcion `issubclass(clase, clase)`:

In [18]:
class B:
    pass
        
class A(B):
    pass

a = A()
b = B()
assert isinstance(a, A) == True
assert isinstance(b, B) == True  
assert isinstance(a, B) == True  # A es un subtipo de B
assert isinstance(b, A) == False  # todos los a son tipos de b, pero no al reves
assert issubclass(A, B) == True


### Sobrecarga de operadores

Se puede, como en C++,
sobreescribir los operadores (operadores aritméticos, acceso por
índices, etc...) mediante una sintaxis especial.

Los métodos y atributos que empiezan y acaban con un doble signo de
subrayado tiene por lo general un significado especial. En algunos
casos es para ocultarlos  o para marcarlos para uso privado, pero
en otros son para usos especiales y tienen significados particulares.

Por ejemplo, si en nuestra clase definimos un método `__len__`,
podemos hacer que las instancias de esa clase puedan ser usadas con la
función `len()`:

In [19]:
class A:
    def __len__(self):
        return 7 # por la cara

a = A()
print(len(a))

7


De igual manera, si a esta clase le añadimos los métodos
`__setitem__` y `__getitem__` podemos hacer que se comporte como
si fuera una contenedor accesible mediante las operaciones de índices.

El siguiente código muestra  una clase que puede ser accedida como si
fuera una lista; si accedemos a las posiciones 0 a la 6 devuelve una
descripción del número en texto (tiene problemas para recordar el 6),
pero si el índice cae fuera de rango, devuelve una descripción
genérica: `Muchos`. Si intentamos modificar sus valores, no da
error, porque hemos definido el método de  asignacion correspondiente,
`__setitem__`, pero como éste no hace nada (la sentencia `pass`,
como su nombre indica, no es muy activa), los intentos de modificarlo
son inútiles:

In [20]:
class A:
    _Tabla = {
        0: 'ninguno',  1: 'uno',     2: 'dos',
        3: 'tres',     4: 'cuatro',  5: 'cinco',
        6: 'umm... seis',
    }
    
    def __len__(self):
        return 7 # por la cara

    def __getitem__(self, index):
        if 0 <= index < 7:
            return self._Tabla[index]
        else:
            return 'Un montón'

    def __setitem__(self, index, value):
        pass

a = A()
assert a[3] == 'tres'
assert a[25] == 'Un montón'
a[4] = 'IV'
assert a[4] == 'cuatro'

**Ejercicio**

> 1) Modifica la clase A para que pueda llegar a 7.
>
> 2) Modifica la clase para que si se intenta modificar el valor, de un error, no como hasta
ahora que no hace nada.

Podemos sobrecargar también operadores algebráicos; por ejemplo,
supongamos que queremos escribir un módulo de álgebra lineal y que
definimos la clase `Vector`; podemos hacer que estos vectores
puedan sumarse o restarse con los operadores `+` y `-`:

In [21]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return 'Vector({0}, {1})'.format(self.x, self.y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

v1 = Vector(2,34)
v2 = Vector(7, 2)
print('Suma:', v1 + v2)
print('Resta:', v1 - v2)

Suma: Vector(9, 36)
Resta: Vector(-5, 32)


Podríamos crear un método para sumar un vector a otro, o una función
independiente para sumar vectores, algo como esto::

```python
v1 = Vector(2, 3)
v2 = Vector(-4, 2)
v3 = suma_vector(v1, v2)
```

Pero es claramente mejor, más legible y bonito, poder hacer:

In [22]:
v1 = Vector(2, 3)
v2 = Vector(-4, 2)
v3 = v1 + v2
print(v3)

Vector(-2, 5)


Para eso definimos los métodos especiales `__add__` y `__sub__`,
con lo cual podemos definir el comportamiento cuando se sumen o resten
dos instancias  de nuesta clase:

Existen muchos métodos especiales; si definimos el método `__str__`,
por ejemplo, será llamado cada vez que Python necesite convertir la
instancia en una cadena de texto (Por ejemplo, para imprimirla por
pantalla, o usando la función `str`). El resultado de este método,
por supuesto, debe ser string. En el apartado 
[*Special method Names*](https://docs.python.org/3/reference/datamodel.html#special-method-names)
dentro del apartado decicado a *Data Model*.